# Trabalho Prático 2
## Realidade Aumentada
#### Introdução à Computação Visual - 2020/1

##### Integrantes:
* Otávio Augusto Silva - 2016006808
* Luiz Henrique de Melo Santos - 2017014464

##### Considerações iniciais:

* Este código foi elaborado primariamente por meio de adaptação do código-fonte fornecido durante a apresentação do professor Erickson R. Nascimento durante a Escola de Verão de Ciência da Computação de 2017, da DCC-UFMG. O vídeo da apresentação pode ser acessado por meio deste [LINK](https://youtu.be/1z0Sga8_RxE).
* Este código foi executado em um ambiente Conda com Python 3.5, em sistema operacional Windows - devido às bibliotecas FreeGlut e Pygame, que apresentaram vários problemas com novas versões do Python, bem como em sistemas como MacOS e Ubuntu.
* O código-fonte _'objloader.py'_ fornecido juntamente com a especificação do trabalho teve de ser alterado em alguns trechos para correção de alguns bugs, que estavam impossibilitando a correta execução da aplicação. Os trechos modificados foram marcados com a tag _'# MODIFICADO'_.
* Não conseguimos de maneira alguma obter o video no formato AVI com todos os quadros de detecção e localização do objeto, bem como o cubo inserido na posicao do objeto.

##### Calibração da câmera

A calibração da câmera foi feita por meio de 7 imagens '.jpg' obtidas a partir do vídeo de entrada original.</br>As imagens utilizadas e os resultados obtidos encontram-se no diretório _'./camera_calibration/'_, nesta mesma pasta.</br>Os parâmetros obtidos na etapa de calibração da câmera foram os seguintes (retirados da saída retornada pelo Octave):

%-- Focal length:
fc = [ 610.879708372179834 ; 567.989563803793203 ];

%-- Principal point:
cc = [ 147.535469452871865 ; 378.898762622333265 ];

%-- Skew coefficient:
alpha_c = 0.000000000000000;

%-- Distortion coefficients:
kc = [ 0.221906437598475 ; -0.144942670158401 ; 0.071131863616536 ; -0.034706440480340 ; 0.000000000000000 ];

%-- Focal length uncertainty:
fc_error = [ 75.155595825974373 ; 60.441536246691285 ];

%-- Principal point uncertainty:
cc_error = [ 25.814231043535663 ; 35.864563206899405 ];

%-- Skew coefficient uncertainty:
alpha_c_error = 0.000000000000000;

%-- Distortion coefficients uncertainty:
kc_error = [ 0.171427317703732 ; 0.152596209865342 ; 0.032279667448496 ; 0.017045499576809 ; 0.000000000000000 ];

%-- Image size:
nx = 578;
ny = 434;


##### Imports para a aplicação (caso não tenha, favor fazer a instalação das seguintes bibliotecas para a correta execução):

In [ ]:
# !pip install pygame
# !pip install PyOpenGL
# !pip install opencv-python

In [2]:
import cv2
import numpy as np

from PIL import Image
from OpenGL.GL import *
from objloader import *
from objloader import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

import matplotlib.pyplot as plt

##### Definição dos arquivos que serão utilizados pela aplicação.

In [3]:
# Imagem do alvo para ser identificado no vídeo de entrada
TARGET = './alvo.jpg'

# Objeto 3D a ser renderizado no vídeo de entrada
OBJECT = "Pikachu.obj"

# Vídeo de entrada a ser analisado, e que receberá a renderização do objeto 3D
VIDEO_INPUT = './tp2-icv-input.mp4'

##### Etapas de "baixo nível": detecção de bordas / detecção do alvo no vídeo

In [4]:
def correlation_coefficient(X, Y):
    '''
    Determina o Coeficiente de Correlacao de Pearson, de acordo com a formula (https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)
    
    INPUTS:
    patch1 - matriz de representacao da imagem do alvo
    patch2 - matriz de representacao da imagem extraida do video de entrada
    
    OUTPUTS:
    0 - imagens nao possuem semelhanca
    product - produto obtidos por meio do calculo da Correlacao
    '''
    
    product = np.mean((X - X.mean()) * (Y - Y.mean()))
    stds = X.std() * Y.std()
    return 0 if stds == 0 else product/stds

In [5]:
def findTargets(img):
    '''
    Encontra os possiveis alvos na imagem do frame atual recebido como entrada
    
    INPUT:
    img - imagem do frame atual extraido do video de entrada
    
    OUTPUT:
    markers - coordenadas dos alvos localizados no frame atual analisado no video
    '''
    
    t_size = 100
    global target

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_gray = cv2.threshold(img_gray, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    markers = []
    edged = cv2.Canny(img_gray, 100, 200)
    cnt, _ = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    for c in cnt:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 12, True)

        if len(approx) == 4:
            c_indices = None
            c_cords = None
            
            dst = np.array([[0, 0], [0, t_size], [t_size, t_size], [t_size, 0]], dtype="float32")
            indices = np.arange(4)
            
            for rotations in range(4):
                src = np.array(approx.reshape(4,2), dtype="float32")
                matrix = cv2.getPerspectiveTransform(src, dst[indices])
                warped = cv2.warpPerspective(img_gray, matrix, (t_size, t_size))
                ret, binaryImg = cv2.threshold(warped, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                
                # Calculo do Coeficiente de Correlacao de Pearson
                dist = correlation_coefficient(binaryImg, target)
                if dist > 0.6:
                    min_dist = dist
                    c_indices = indices
                    c_cords = src
                    break
                    
                target = cv2.rotate(target, cv2.ROTATE_90_CLOCKWISE)
            if c_cords is not None:
                markers.append((c_cords, c_indices))
    return markers

##### Etapas visuais: início da framework / execução do vídeo / renderização do objeto 3D

In [6]:
def initOpenGL(img):
    '''
    Inicia as instâncias do OpenGL, bem como define os parametros a serem utilizados por ele
    
    INPUT
    img - frame atual analisado, do video
    '''
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    
    background = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)
    height, width, channels = background.shape
    background = np.frombuffer(background.tostring(), dtype=background.dtype, count=height * width * channels)
    background.shape = (height, width, channels)
    
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, pikachu_tex)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glDepthMask(GL_FALSE)
    glMatrixMode(GL_PROJECTION)
    
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    glMatrixMode(GL_MODELVIEW)
    glBindTexture(GL_TEXTURE_2D, pikachu_tex)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glPushMatrix()
    
    glBegin(GL_QUADS)
    glTexCoord2i(0, 0); glVertex2i(0, 0)
    glTexCoord2i(1, 0); glVertex2i(width, 0)
    glTexCoord2i(1, 1); glVertex2i(width, height)
    glTexCoord2i(0, 1); glVertex2i(0, height)
    glEnd()
    
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)
    glDepthMask(GL_TRUE)
    glDisable(GL_TEXTURE_2D)

##### Etapas escalonadoras: callback para execucao continua do programa em cada um dos frame do video

In [7]:
def applicationInit(width=640, height=480): 
    '''
    Promove a configuracao dos parametros que serao utilizados pela aplicacao,
    tal como os parametros da matriz e o tamanho das imagens que serao analizadas
    durante a execucao
    
    INPUT:
    width - largura da imagem
    height - altura da imagem
    '''
    
    glutInit()
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
    glutInitWindowSize(width, height)
    glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
    window = glutCreateWindow(b'Trabalho Pratico 02 - Realidade Aumentada')
    glMatrixMode(GL_PROJECTION)

    global video, cameraMatrix, distCoeffs, obj, pikachu_tex, target
    
    '''
    ** ATENCAO **
    Os valores da calibracao foram alterados de ultima hora para correcao,
    pois os exibidos anteriormente estavam desregulando consideravelemnte
    a posicao de renderizacao do objeto 3D - os novos valores foram obtidos
    apos uma nova calibracao, e os resultados melhorados consideravelmente
    '''
    fx, fy = 1175.32408, 1180.34557
    cx, cy = 933.43195, 544.17464
    cameraMatrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])  # Matriz de parametros intrinsecos
    distCoeffs = np.array([0.07378, -0.04224, -0.00111, -0.00524, 0.00000])  # Coeficientes de distorcao
    
    fovy = 60
    aspect = width/height
    gluPerspective(fovy, aspect, 0.01, 100.0)
    
    target = cv2.imread(TARGET)
    target = cv2.resize(target, (100, 100), interpolation = cv2.INTER_AREA)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    _, target = cv2.threshold(target, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    obj = OBJ(OBJECT, swapyz=True)
    glEnable(GL_TEXTURE_2D)
    pikachu_tex = glGenTextures(1)
    video = cv2.VideoCapture(VIDEO_INPUT)

In [8]:
def glutCallback():
    '''
    Permite que todos os frame do video sejam analizados
    e que os objetos possam ser renderizados em cada um deles
    '''
    
    glEnable(GL_TEXTURE_2D)
    glMatrixMode(GL_MODELVIEW)
    
    _, img = video.read()
    initOpenGL(img)
    targets = findTargets(img)
    
    for rect, index in targets:
        imagePoints = np.array(rect, dtype="float32")
        objectPoints = np.array([[-1, -1, 1], [ 1, -1, 1], [ 1,  1, 1], [-1,  1, 1]], dtype="float32")
        _, rvecs, tvecs = cv2.solvePnP(objectPoints[index], imagePoints, cameraMatrix, distCoeffs)
        rot_mat, _ = cv2.Rodrigues(rvecs)
        mat = np.transpose(np.array([
            [ rot_mat[0][0],  rot_mat[0][1],  rot_mat[0][2],  tvecs[0]], 
            [-rot_mat[1][0], -rot_mat[1][1], -rot_mat[1][2], -tvecs[1]], 
            [-rot_mat[2][0], -rot_mat[2][1], -rot_mat[2][2], -tvecs[2]], 
            [           0.0,            0.0,            0.0,       1.0]]))
        
        glBindTexture(GL_TEXTURE_2D, pikachu_tex)
        glEnable(GL_TEXTURE_2D)
        glLoadMatrixd(mat)
        glCallList(obj.gl_list)
        
    glutSwapBuffers()

In [9]:
if __name__ == '__main__':
    '''
    Funcao principal para inicio da aplicacao de Realidade Aumentada
    '''
    
    applicationInit()
    glutDisplayFunc(glutCallback)
    glutIdleFunc(glutPostRedisplay)
    glutMainLoop()
    
    video.release()